In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize

In [2]:
%load_ext sql
%sql sqlite://

'Connected: @None'

# Pharmacy Claim Analysis

In [3]:
%%sql
drop table if exists claims;
CREATE TABLE claims (
  Transaction_Id numeric,
  Prescription_Nbr numeric,
  Date_Of_Service date,
  NDC_Code numeric,
  Drug_Name varchar,
  Fill_Number numeric,
  Patient_Id numeric,
  Quantity numeric,
  Billed_Amount numeric,
  Paid_Amount numeric,
  Copay numeric,
  Paid_Date date
);

INSERT INTO claims (Transaction_Id,
  Prescription_Nbr,
  Date_Of_Service,
  NDC_Code,
  Drug_Name,
  Fill_Number,
  Patient_Id,
  Quantity,
  Billed_Amount,
  Paid_Amount,
  Copay,
  Paid_Date)
  
VALUES
(161025154809066001, 000001628638, '20131008', 0074433902, 'HUMIRA40MG/0.8', 0, 20020197000, 2, 143.89, 143.04, 0.85, '20131031'),
(161027598362198001, 000001628638, '20131008', 0074433902, 'HUMIRA40MG/0.8', 0, 20020197000, -2, -143.89, -143.04, -0.85, '20131031'),
(151594253779211001, 000001628638, '20131008', 0074433902, 'HUMIRA40MG/0.8', 1, 20020197000, 2, 143.89, 143.04, 0.85, '20131031'),
(161055441486135001, 000001628638, '20131008', 0074433902, 'HUMIRA40MG/0.8', 0, 20020197000, 2, 143.89, 143.04, 0.85, '20131110'),
(161027598362198001, 000001628638, '20131008', 0074433902, 'HUMIRA40MG/0.8', 0, 20020197000, -2, -143.89, -143.04, -0.85, '20131031');

 * sqlite://
Done.
Done.
5 rows affected.


[]

In [4]:
%%sql
select * from claims;

 * sqlite://
Done.


Transaction_Id,Prescription_Nbr,Date_Of_Service,NDC_Code,Drug_Name,Fill_Number,Patient_Id,Quantity,Billed_Amount,Paid_Amount,Copay,Paid_Date
161025154809066001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,2,143.89,143.04,0.85,20131031
161027598362198001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,-2,-143.89,-143.04,-0.85,20131031
151594253779211001,1628638,20131008,74433902,HUMIRA40MG/0.8,1,20020197000,2,143.89,143.04,0.85,20131031
161055441486135001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,2,143.89,143.04,0.85,20131110
161027598362198001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,-2,-143.89,-143.04,-0.85,20131031


In [5]:
%%sql

/*a. Identify duplicates*/

select Transaction_Id, Prescription_nbr, NDC_Code, Drug_Name,
Fill_Number, Patient_Id, Quantity,Billed_Amount, Paid_Amount, Copay,
Paid_Date, count(*) as row_count
from claims
group by 1,2,3,4,5,6,7,8,9,10,11
having count(*)>1;

 * sqlite://
Done.


Transaction_Id,Prescription_Nbr,NDC_Code,Drug_Name,Fill_Number,Patient_Id,Quantity,Billed_Amount,Paid_Amount,Copay,Paid_Date,row_count
161027598362198001,1628638,74433902,HUMIRA40MG/0.8,0,20020197000,-2,-143.89,-143.04,-0.85,20131031,2


In [6]:
%%sql

/*b. Identify paid vs reversed vs adjusted claims.

Assumption made that though transaction IDs are the same, they can be separate events in time (at hour level).*/

select*, 
case when Copay + Paid_Amount = Billed_Amount then 'paid'
     when Paid_Amount <0 then 'reversed' 
     when (Copay + Paid_Amount != Billed_Amount) and Paid_Amount>0 then 'adjusted'
     end as claim_type
from(select*
from claims
order by paid_date) sub;

 * sqlite://
Done.


Transaction_Id,Prescription_Nbr,Date_Of_Service,NDC_Code,Drug_Name,Fill_Number,Patient_Id,Quantity,Billed_Amount,Paid_Amount,Copay,Paid_Date,claim_type
161025154809066001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,2,143.89,143.04,0.85,20131031,paid
161027598362198001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,-2,-143.89,-143.04,-0.85,20131031,paid
151594253779211001,1628638,20131008,74433902,HUMIRA40MG/0.8,1,20020197000,2,143.89,143.04,0.85,20131031,paid
161027598362198001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,-2,-143.89,-143.04,-0.85,20131031,paid
161055441486135001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,2,143.89,143.04,0.85,20131110,paid


In [7]:
%%sql

/*c. Identify which record is the latest claim among related prescriptions.*/

select a.Transaction_Id
from claims a
where a.Date_Of_Service =
(select max(b.Date_of_Service) from claims b where b.Prescription_Nbr= a.Prescription_Nbr group by b.Prescription_Nbr)
;

 * sqlite://
Done.


Transaction_Id
161025154809066001
161027598362198001
151594253779211001
161055441486135001
161027598362198001


In [8]:
%%sql

/*d.Identify records relating to the same prescription (logical key).*/

select*
from claims c1
join claims c2
on c1.Prescription_Nbr = c2.Prescription_Nbr;

 * sqlite://
Done.


Transaction_Id,Prescription_Nbr,Date_Of_Service,NDC_Code,Drug_Name,Fill_Number,Patient_Id,Quantity,Billed_Amount,Paid_Amount,Copay,Paid_Date,Transaction_Id_1,Prescription_Nbr_1,Date_Of_Service_1,NDC_Code_1,Drug_Name_1,Fill_Number_1,Patient_Id_1,Quantity_1,Billed_Amount_1,Paid_Amount_1,Copay_1,Paid_Date_1
161025154809066001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,2,143.89,143.04,0.85,20131031,151594253779211001,1628638,20131008,74433902,HUMIRA40MG/0.8,1,20020197000,2,143.89,143.04,0.85,20131031
161025154809066001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,2,143.89,143.04,0.85,20131031,161025154809066001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,2,143.89,143.04,0.85,20131031
161025154809066001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,2,143.89,143.04,0.85,20131031,161027598362198001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,-2,-143.89,-143.04,-0.85,20131031
161025154809066001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,2,143.89,143.04,0.85,20131031,161027598362198001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,-2,-143.89,-143.04,-0.85,20131031
161025154809066001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,2,143.89,143.04,0.85,20131031,161055441486135001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,2,143.89,143.04,0.85,20131110
161027598362198001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,-2,-143.89,-143.04,-0.85,20131031,151594253779211001,1628638,20131008,74433902,HUMIRA40MG/0.8,1,20020197000,2,143.89,143.04,0.85,20131031
161027598362198001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,-2,-143.89,-143.04,-0.85,20131031,161025154809066001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,2,143.89,143.04,0.85,20131031
161027598362198001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,-2,-143.89,-143.04,-0.85,20131031,161027598362198001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,-2,-143.89,-143.04,-0.85,20131031
161027598362198001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,-2,-143.89,-143.04,-0.85,20131031,161027598362198001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,-2,-143.89,-143.04,-0.85,20131031
161027598362198001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,-2,-143.89,-143.04,-0.85,20131031,161055441486135001,1628638,20131008,74433902,HUMIRA40MG/0.8,0,20020197000,2,143.89,143.04,0.85,20131110


In [9]:
%%sql
 
/*e.  Other interesting analysis!

(1) Per Patient, Days since Service Date, Cumulative Insurance payment to pay date, Order by Pay Date*/

select Patient_Id, Paid_Date-Date_Of_Service as Days_Since_Service_Date, sum(Paid_Amount) over (partition by Paid_Date order by Date_Of_Service) as Insurance_Paid_Amount
from claims
group by 1,2, Paid_Amount, Paid_Date, Date_Of_Service;

 * sqlite://
Done.


Patient_Id,Days_Since_Service_Date,Insurance_Paid_Amount
20020197000,23,0.0
20020197000,23,0.0
20020197000,102,143.04


In [10]:
%%sql

/*e.  Other interesting analysis!

(2)  For each Patient_Id, each Date_Of_Service, drug type count and quantity for each drug type.*/

select Patient_Id, Date_Of_Service,
sum(case when drug_name = 'HUMIRA40MG/0.8' then 1 else 0 end) as HUMIRA_count, 
sum(case when drug_name = 'HUMIRA40MG/0.8' then Quantity else 0 end) as  HUMIRA_qty
from claims
group by 1,2

 * sqlite://
Done.


Patient_Id,Date_Of_Service,HUMIRA_count,HUMIRA_qty
20020197000,20131008,5,2


# 2. Python Questions

In [11]:
data = [{'member_id': {'XXXX'}, 
         'claims': [{'date_of_service': '20190101', 'procedures': [{'code':"11111", 'price':10.45},
                                                                  {'code':"22222", 'price':11.45},
                                                                  {'code':"33333", 'price':12.45}]},
                    {'date_of_service': '20190102', 'procedures': [{'code':"11111", 'price':10.45},
                                                                  {'code':"22222", 'price':11.45},
                                                                  {'code':"33333", 'price':12.45}]},
                    {'date_of_service': '20190103', 'procedures': [{'code':"11111", 'price':10.45},
                                                                  {'code':"22222", 'price':11.45},
                                                                  {'code':"33333", 'price':12.45}]}]},
        
       
         
         
         {'member_id': {'YYYY'}, 
         'claims': [{'date_of_service': '20190101', 'procedures': [{'code':"11111", 'price':10.45},
                                                                  {'code':"22222", 'price':11.45},
                                                                  {'code':"33333", 'price':12.45}]},
                    {'date_of_service': '20190102', 'procedures': [{'code':"11111", 'price':10.45},
                                                                  {'code':"22222", 'price':11.45},
                                                                  {'code':"33333", 'price':12.45}]},
                    {'date_of_service': '20190103', 'procedures': [{'code':"11111", 'price':10.45},
                                                                  {'code':"22222", 'price':11.45},
                                                                  {'code':"33333", 'price':12.45}]}]}]

In [12]:
#member_id is retrieved.
claims_data = json_normalize(data=data, record_path='claims', 
                            meta=['member_id'])
claims_data

,date_of_service,procedures,member_id
0,20190101,"[{'code': '11111', 'price': 10.45}, {'code': '...",{XXXX}
1,20190102,"[{'code': '11111', 'price': 10.45}, {'code': '...",{XXXX}
2,20190103,"[{'code': '11111', 'price': 10.45}, {'code': '...",{XXXX}
3,20190101,"[{'code': '11111', 'price': 10.45}, {'code': '...",{YYYY}
4,20190102,"[{'code': '11111', 'price': 10.45}, {'code': '...",{YYYY}
5,20190103,"[{'code': '11111', 'price': 10.45}, {'code': '...",{YYYY}


## Member-level JSON documents.

In [13]:
#Reordering column names.
claims_data1 = claims_data[['member_id','date_of_service', 'procedures']]
claims_data1

,member_id,date_of_service,procedures
0,{XXXX},20190101,"[{'code': '11111', 'price': 10.45}, {'code': '..."
1,{XXXX},20190102,"[{'code': '11111', 'price': 10.45}, {'code': '..."
2,{XXXX},20190103,"[{'code': '11111', 'price': 10.45}, {'code': '..."
3,{YYYY},20190101,"[{'code': '11111', 'price': 10.45}, {'code': '..."
4,{YYYY},20190102,"[{'code': '11111', 'price': 10.45}, {'code': '..."
5,{YYYY},20190103,"[{'code': '11111', 'price': 10.45}, {'code': '..."
